## Sympgpt
This is Small language model build on gpt2 by huggingface on custom dataset for prediction and Q/A of disease and symptoms.

Dataset Link: https://huggingface.co/datasets/prognosis/symptoms_disease_v1/viewer/default/train?p=2

In [2]:
# Installing the required libraries
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [3]:
# loading the libraries
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [4]:
# Load data set from huggingface
train_dataset = load_dataset("prognosis/symptoms_disease_v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction', 'input', 'reference'],
        num_rows: 10110
    })
})

In [6]:
# Convert to a pandas dataframe
updated_data = [{'instruction': item['instruction'], 'output': item['output']} for item in train_dataset['train']]
df = pd.DataFrame(updated_data)

In [7]:
df.head(5)

,instruction,output
0,What are the symptoms of hypertensive disease?,The following are the symptoms of hypertensive...
1,I am having the following symptoms: pain ches...,The symptoms listed indicates that the patient...
2,What are the symptoms of diabetes?,The following are the symptoms of diabetes: po...
3,"I am having the following symptoms: polyuria, ...",The symptoms listed indicates that the patient...
4,What are the symptoms of depressive disorder?,The following are the symptoms of depressive d...


In [47]:
df['output'] = df['output'].apply(lambda x: ', '.join(x.split(', ')))
display(df.head())

,instruction,output
0,What are the symptoms of hypertensive disease?,The following are the symptoms of hypertensive...
1,I am having the following symptoms: pain ches...,The symptoms listed indicates that the patient...
2,What are the symptoms of diabetes?,The following are the symptoms of diabetes: po...
3,"I am having the following symptoms: polyuria, ...",The symptoms listed indicates that the patient...
4,What are the symptoms of depressive disorder?,The following are the symptoms of depressive d...


In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [11]:
# If you have an NVIDIA GPU attached, use 'cuda'
if torch.cuda.is_available():
    device = torch.device('cuda')
print(device)

cuda


In [12]:
# The tokenizer turns texts to numbers (and vice-versa)
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# The transformer
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
# gpt2 model architecture
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [16]:
df.describe()

,instruction,output
count,10110,10110
unique,615,615
top,What are the symptoms of Chicken pox?,The symptoms listed indicates that the patient...
freq,120,120


In [17]:
# Dataset Preparation, taken from the internet

class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x

In [18]:
data_sample = LanguageDataset(df, tokenizer)

In [19]:
data_sample

In [20]:
# Create train, valid
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])

In [31]:
# Model params
BATCH_SIZE = 10
num_epochs = 25

In [32]:
# Make the iterators
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [33]:
# Training parameters
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [34]:
# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [35]:
# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])

## Training the model

In [36]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")

Training Epoch 1/25 Batch Size: 10, Transformer: distilgpt2: 100%|██████████| 809/809 [03:16<00:00,  4.12it/s, Training Loss=0.0577]
Validation Epoch 1/25: 100%|██████████| 203/203 [00:14<00:00, 13.67it/s, Validation Loss=0.326]


Epoch: 1, Validation Loss: 0.08090632408857346


Training Epoch 2/25 Batch Size: 10, Transformer: distilgpt2: 100%|██████████| 809/809 [03:14<00:00,  4.16it/s, Training Loss=0.0404]
Validation Epoch 2/25: 100%|██████████| 203/203 [00:14<00:00, 13.64it/s, Validation Loss=0.353]


Epoch: 2, Validation Loss: 0.07463762164115906


Training Epoch 3/25 Batch Size: 10, Transformer: distilgpt2: 100%|██████████| 809/809 [03:14<00:00,  4.17it/s, Training Loss=0.0629]
Validation Epoch 3/25: 100%|██████████| 203/203 [00:15<00:00, 13.53it/s, Validation Loss=0.349]


...

Epoch: 23, Validation Loss: 0.06480598449707031


Training Epoch 24/25 Batch Size: 10, Transformer: distilgpt2: 100%|██████████| 809/809 [03:13<00:00,  4.18it/s, Training Loss=0.0424]
Validation Epoch 24/25: 100%|██████████| 203/203 [00:14<00:00, 13.53it/s, Validation Loss=0.723]


Epoch: 24, Validation Loss: 0.06577666848897934


Training Epoch 25/25 Batch Size: 10, Transformer: distilgpt2: 100%|██████████| 809/809 [03:13<00:00,  4.18it/s, Training Loss=0.0448]
Validation Epoch 25/25: 100%|██████████| 203/203 [00:14<00:00, 13.58it/s, Validation Loss=0.642]

Epoch: 25, Validation Loss: 0.06437286734580994


## Loading the model and inferencing the output

In [41]:
input_str = "What are the symptoms of chicken pox?"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=216,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What are the symptoms of chicken pox? | The following are the symptoms of Chicken pox: itching, skin rash, fatigue, lethargy, high fever, headache, loss of appetite, mild fever, swelled lymph nodes, malaise, red spots over body


### generating the response on multiple inputs and comparing with the real data

In [44]:
while True:
  input_str = input("Enter your question: ")
  input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

  output = model.generate(
    input_ids,
    max_length=216,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
  )

  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  print(decoded_output)
  print("\n")

Enter your question: What is the symptoms of depression?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the symptoms of depression? | The following are the symptoms of depressive disorder: feeling  suicidal, hallucinations  auditory, irritable  mood, verbal  auditory hallucinations, feeling  depressed, decompensation, scar tissue, blackout, loose associations


Enter your question: What are the symptoms of ulcer?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What are the symptoms of ulcer? | The following are the symptoms of ulcer  peptic: pain abdominal, paraparesis, nausea, vomiting, polymyalgia, out of breath, pain chest, hemiplegia, gurgle, hunger, apyrexial, nervousness


Enter your question: I have chest pain and headache. what are the diseases I may have?


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How aeroplane fly? | The following are the symptoms of migraine  disorders: ambidexterity, vomiting, dizziness, puffy face and eyes, enlarged thyroid, brittle nails, swollen extremeties, depression, irritability, abnormal menstruation


Enter your question: exit


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


exit am having the following symptoms: fatigue, weight gain, cold hands and feets, mood swings, lethargy, dizziness, puffy face and eyes, enlarged thyroid, brittle nails, swollen extremeties, depression, irritability, abnormal menstruation. What could be the disease? | The symptoms listed indicates that the patient is dealing with Hypothyroidism.




KeyboardInterrupt: Interrupted by user

In [38]:
torch.save(model, 'Sympgpt.pt')